In [2]:
from datasets import Dataset, DatasetDict
from PIL import Image
import pillow_heif

import pandas as pd
from openai import OpenAI
import requests
import io
import base64
import tqdm
from apify_client import ApifyClient

import os
from dotenv import load_dotenv
load_dotenv()

APIFY_TOKEN = os.getenv('APIFY_TOKEN')

In [3]:
client = ApifyClient(APIFY_TOKEN)

In [4]:
run_input = {
    "directUrls": [
        "https://www.instagram.com/secso.posting", 
        "https://www.instagram.com/abuelita_eliana",
        "https://www.instagram.com/kedocreisi.jpg",
        "https://www.instagram.com/humorymemes.cl"
    ],
    "resultsType": "posts",
    "resultsLimit": 100,
    "searchType": "hashtag",
    "searchLimit": 6,
    "addParentData": False,
}

run = client.actor("shu8hvrXbJbY3Eb9W").call(run_input=run_input)

In [5]:
dataset = []
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    dataset.append(item)

df = pd.DataFrame(dataset)

In [6]:
df.head()

,inputUrl,id,type,shortCode,caption,hashtags,mentions,url,commentsCount,firstComment,...,videoUrl,videoViewCount,videoPlayCount,productType,videoDuration,musicInfo,taggedUsers,coauthorProducers,locationName,locationId
0,https://www.instagram.com/secso.posting,3275080335421651217,Image,C1zbDaDrLkR,vivimos en una torta,[],[],https://www.instagram.com/p/C1zbDaDrLkR/,66,Tenía q ver está wea justo antes de dormir,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.instagram.com/secso.posting,3476553986935123047,Image,DA_M255vGxn,en realidad no somos más que semen que se desa...,[],[],https://www.instagram.com/p/DA_M255vGxn/,217,Teamokingcrimson,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.instagram.com/secso.posting,3639948755609086001,Image,DKDshuitGgx,Jesús era comunista,[],[],https://www.instagram.com/p/DKDshuitGgx/,13,"mmm nope, que yo sepa yo no ando caminando con...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.instagram.com/secso.posting,3289569404507719420,Image,C2m5fHLrQb8,gato samurai xixitumare,[],[],https://www.instagram.com/p/C2m5fHLrQb8/,100,Literal soy ese,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.instagram.com/secso.posting,3638947609662909386,Image,DKAI5IuNuvK,,[],[],https://www.instagram.com/p/DKAI5IuNuvK/,1,@daniel_fdr__,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
diccionario_posts = {}
df_final = []
i = 1
for url in df['url']:
    print(f"Analizando Post {i}: {url}...")
    comentarios = []
    Input_Schema = {
    "addParentData": False,
    "directUrls": [url],
    "enhanceUserSearchWithFacebookPage": False,
    "isUserReelFeedURL": False,
    "isUserTaggedFeedURL": False,
    "resultsLimit": 6,
    "resultsType": "posts",
    "searchLimit": 6,
    "searchType": "user"
    }

    run = client.actor("shu8hvrXbJbY3Eb9W").call(run_input=Input_Schema)
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        comentarios.append(item)
    comentarios_df = pd.DataFrame(comentarios)
    df_final.append(comentarios_df)
    i += 1

df_final = pd.concat(df_final)

Analizando Post 1: https://www.instagram.com/p/C1zbDaDrLkR/...
Analizando Post 2: https://www.instagram.com/p/DA_M255vGxn/...
Analizando Post 3: https://www.instagram.com/p/DKDshuitGgx/...
Analizando Post 4: https://www.instagram.com/p/C2m5fHLrQb8/...
Analizando Post 5: https://www.instagram.com/p/DKAI5IuNuvK/...
Analizando Post 6: https://www.instagram.com/p/DJ7YonZx2A2/...
Analizando Post 7: https://www.instagram.com/p/DJrk3kiNNJF/...
Analizando Post 8: https://www.instagram.com/p/DJ0VKjcNh0U/...
Analizando Post 9: https://www.instagram.com/p/DHGZ_3MuGHa/...
Analizando Post 10: https://www.instagram.com/p/DGBHb59ucps/...
Analizando Post 11: https://www.instagram.com/p/DF0vAncSOgS/...
Analizando Post 12: https://www.instagram.com/p/DKIyRifOKcS/...
Analizando Post 13: https://www.instagram.com/p/DKF8vvby9OV/...
Analizando Post 14: https://www.instagram.com/p/DKAmVCnSptA/...
Analizando Post 15: https://www.instagram.com/p/DJ-WlLOyked/...
Analizando Post 16: https://www.instagram.com/p/D

In [8]:
publicaciones_dict = {}

for idx, publicacion in df_final.iterrows():
    type_publicaion = publicacion['type']
    if type_publicaion != 'Image':
        print(f"Publicación es un Reel.")
        continue
    url = publicacion['displayUrl']
    comentarios = publicacion['latestComments']
    comentarios_list = []
    for comentario in comentarios:
        texto_comentario = comentario['text']
        comentarios_list.append(texto_comentario)
    publicaciones_dict[url] = comentarios_list

Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es un Reel.
Publicación es u

In [9]:
publicaciones_dict

{'https://scontent-atl3-1.cdninstagram.com/v/t51.29350-15/417195221_1044056883316150_4701265457306111522_n.webp?stp=dst-jpg_e35_tt6&_nc_ht=scontent-atl3-1.cdninstagram.com&_nc_cat=110&_nc_oc=Q6cZ2QFsjmCPDrq0S1nS6oymHZEd7tKZdGWnNwI8AhJchWuaHHP1RZK2KreOxa0E5UQvO3TyS02nlGGfTTKXsKsdH-Up&_nc_ohc=KrhUNEm8BX0Q7kNvwEfmy6Q&_nc_gid=I_Sl7M3LX5GvHDEPMBu0qw&edm=APs17CUBAAAA&ccb=7-5&oh=00_AfKfevv8UoZyjgF5I-N3Gh1kPWuxmA74ertLux7KPDQwiQ&oe=683B026A&_nc_sid=10d13b': ['Tenía q ver está wea justo antes de dormir',
  'La wea pulenta',
  '😂😂😂😂',
  'Un dia me iba matar y me di cuenta que era una torta tresleches',
  '😂😂😂😂',
  'ya lo comprobe, no soy una torta (ayuda)',
  'Objeto o pastel',
  'No soy un pastel 😭😭😭😭'],
 'https://scontent-ord5-3.cdninstagram.com/v/t51.29350-15/462812857_1071683797818213_8119219543913201621_n.webp?stp=dst-jpg_e35_s1080x1080_tt6&_nc_ht=scontent-ord5-3.cdninstagram.com&_nc_cat=109&_nc_oc=Q6cZ2QGuAczNLxnE0luqlzq9nNGV_crobYQ9e92AE2bv038D2f-K7BEM4xGWkQmz36OSKzXpW48wGQ5QoOD1wYAeeVo3&

In [10]:
pillow_heif.register_heif_opener()

os.makedirs('../data/raw', exist_ok=True)

rows = []
i= 1
for url, comments in publicaciones_dict.items():
    print(f"Post {i} de {len(publicaciones_dict)}")
    print(f"Intentando descargar {url}...")
    # Extraer nombre base y genera nombre .jpg
    original_name = os.path.basename(url.split('?', 1)[0])
    root, _ = os.path.splitext(original_name)
    jpg_name = f"{root}.jpg"
    jpg_path = os.path.join('../data/raw', jpg_name)

    # Descargar la imagen HEIC en memoria
    try:
        resp = requests.get(url)
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar la imagen: {e}")
        continue
    resp.raise_for_status()
    img_bytes = io.BytesIO(resp.content)

    # Abrir y convertir a JPEG
    img = Image.open(img_bytes)
    img = img.convert('RGB')
    img.save(jpg_path, 'JPEG', quality=95)

    # Unimos comentarios
    joined_comments = ' '.join(comments)

    rows.append({
        'ruta': jpg_path,
        'comentarios': joined_comments,
        'url': url
    })
    i += 1

dataset_publicaciones = pd.DataFrame(rows)

Post 1 de 326
Intentando descargar https://scontent-atl3-1.cdninstagram.com/v/t51.29350-15/417195221_1044056883316150_4701265457306111522_n.webp?stp=dst-jpg_e35_tt6&_nc_ht=scontent-atl3-1.cdninstagram.com&_nc_cat=110&_nc_oc=Q6cZ2QFsjmCPDrq0S1nS6oymHZEd7tKZdGWnNwI8AhJchWuaHHP1RZK2KreOxa0E5UQvO3TyS02nlGGfTTKXsKsdH-Up&_nc_ohc=KrhUNEm8BX0Q7kNvwEfmy6Q&_nc_gid=I_Sl7M3LX5GvHDEPMBu0qw&edm=APs17CUBAAAA&ccb=7-5&oh=00_AfKfevv8UoZyjgF5I-N3Gh1kPWuxmA74ertLux7KPDQwiQ&oe=683B026A&_nc_sid=10d13b...
Post 2 de 326
Intentando descargar https://scontent-ord5-3.cdninstagram.com/v/t51.29350-15/462812857_1071683797818213_8119219543913201621_n.webp?stp=dst-jpg_e35_s1080x1080_tt6&_nc_ht=scontent-ord5-3.cdninstagram.com&_nc_cat=109&_nc_oc=Q6cZ2QGuAczNLxnE0luqlzq9nNGV_crobYQ9e92AE2bv038D2f-K7BEM4xGWkQmz36OSKzXpW48wGQ5QoOD1wYAeeVo3&_nc_ohc=VTAK_z1_S24Q7kNvwHZaQBa&_nc_gid=5gyjwS4zmZMAJtsNkvNU9A&edm=APs17CUBAAAA&ccb=7-5&oh=00_AfI7CxMHuqTd2cnpke5vLLJ5AEnn7EuD60RUECvrFASakw&oe=683B1931&_nc_sid=10d13b...
Post 3 de 326

In [11]:
dataset_publicaciones.to_csv('../data/dataset_correction.csv', index=False)

In [12]:
dataset_publicaciones = pd.read_csv('../data/dataset_correction.csv')

In [13]:
dataset_publicaciones.shape

(324, 3)

In [26]:
client = OpenAI()

def explain_meme_chileno(ruta, comentarios, max_comments=20):
    system = """
    Eres un chileno nacido y criado en Santiago. Explica de manera clara qué significa este meme obtenido de una publicación de Instagram en chileno con la cercanía y la buena onda de un amigo, usando un español chileno coloquial cuando sea natural. 
    Siempre responde solo con la explicación del meme. Agrega referencias, ejemplos o explicaciones breves cuando sea necesario para que se entienda bien, pero mantén un tono claro, relajado y chileno sin exagerar.
    """
    user = f"Contexto Comentarios de la Publicación:\n{comentarios}\n\nAhora, mira la imagen adjunta y describe hablando en chileno sin exagerar. No debes citar ni mencionar los comentarios."

    with open(ruta, "rb") as img:
        img_data = img.read()

    b64 = base64.b64encode(img_data).decode("utf-8")

    messages = [
    {"role": "system", "content": system},
    {
            "role": "user",
            "content": [
                {"type": "text", "text": user},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{b64}"
                    }
                }
            ]
        }
    ]

    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )
    return resp.choices[0].message.content.strip()

In [ ]:
records = []

In [27]:
for idx, entry in tqdm.tqdm(dataset_publicaciones.iloc[303:].iterrows(), initial=303, total=len(dataset_publicaciones) - 303):
    img = Image.open(entry["ruta"])
    ruta = entry["ruta"]
    url = entry["url"]
    comentarios = entry["comentarios"]
    answer = explain_meme_chileno(ruta, comentarios)
    records.append({
        "image": img,
        "url": url,
        "ruta": ruta,
        "question": "¿Cómo describirías este meme en chileno?",
        "multiple_choice_answer": answer
    })

324it [02:11,  6.29s/it]


In [28]:
len(records)

323

In [31]:
df_publicacion_correcion = pd.DataFrame(records)
df_publicacion_correcion.to_excel('../data/dataset_correction.xlsx', index=False)

In [32]:
df_publicacion_correcion

,image,url,ruta,question,multiple_choice_answer
0,<PIL.JpegImagePlugin.JpegImageFile image mode=...,https://scontent-atl3-1.cdninstagram.com/v/t51...,../data/raw/417195221_1044056883316150_4701265...,¿Cómo describirías este meme en chileno?,El meme muestra a un mono reflexionando mientr...
1,<PIL.JpegImagePlugin.JpegImageFile image mode=...,https://scontent-ord5-3.cdninstagram.com/v/t51...,../data/raw/462812857_1071683797818213_8119219...,¿Cómo describirías este meme en chileno?,Este meme muestra a Jesús flotando en el cielo...
2,<PIL.JpegImagePlugin.JpegImageFile image mode=...,https://scontent-iad3-1.cdninstagram.com/v/t51...,../data/raw/500690227_17963797841914567_711132...,¿Cómo describirías este meme en chileno?,Este meme juega con la idea de que las enseñan...
3,<PIL.JpegImagePlugin.JpegImageFile image mode=...,https://scontent-yyz1-1.cdninstagram.com/v/t51...,../data/raw/422101662_427862399574713_71392327...,¿Cómo describirías este meme en chileno?,La imagen es un meme de un gato que se ve medi...
4,<PIL.JpegImagePlugin.JpegImageFile image mode=...,https://instagram.fuio32-1.fna.fbcdn.net/v/t51...,../data/raw/491452569_17963561432914567_208246...,¿Cómo describirías este meme en chileno?,"¡Wena, cabro! Mira, el chiste está en que en e..."
...,...,...,...,...,...
318,<PIL.JpegImagePlugin.JpegImageFile image mode=...,https://scontent-iad3-2.cdninstagram.com/v/t51...,../data/raw/470973571_1108056314187111_8161240...,¿Cómo describirías este meme en chileno?,En la imagen hay un par de magos sentados en u...
319,<PIL.JpegImagePlugin.JpegImageFile image mode=...,https://instagram.fknu1-3.fna.fbcdn.net/v/t51....,../data/raw/472205812_1770846700395391_1621148...,¿Cómo describirías este meme en chileno?,El meme muestra a un personaje de un videojueg...
320,<PIL.JpegImagePlugin.JpegImageFile image mode=...,https://scontent-ord5-3.cdninstagram.com/v/t51...,../data/raw/470471491_17945219873914567_703002...,¿Cómo describirías este meme en chileno?,Este meme refleja perfecto la sensación de cho...
321,<PIL.JpegImagePlugin.JpegImageFile image mode=...,https://scontent-atl3-3.cdninstagram.com/v/t51...,../data/raw/469894424_17944890755914567_793627...,¿Cómo describirías este meme en chileno?,"Este meme muestra la portada del álbum ""Louder..."
